In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets

from torch.utils.data import DataLoader

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
# parameters
learning_rate = 0.00005
epochs = 20
batch_size = 8

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])

train_data = dsets.ImageFolder(root='/content/drive/Othercomputers/내 노트북/인공지능/모두를 위한 딥러닝/custom_data/train_data', transform=trans)

In [ ]:
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2) # num_workers가 뭐야

In [ ]:
# 데이터 모양 조사
for x,y in data_loader:
    print(x.shape)
    print(y)
    break

torch.Size([8, 3, 64, 128])
tensor([0, 0, 0, 0, 0, 0, 0, 0])


In [5]:
# 모델 설계
# [batch_size, channels, height, width] 미리 계산
input = torch.Tensor(8,3,64,128)
layer1 = nn.Conv2d(3,6,5)
layer2 = nn.Conv2d(6,16,5)
maxpool = nn.MaxPool2d(2)

out1 = layer1(input)
out2 = maxpool(out1)
out3 = layer2(out2)
out4 = maxpool(out3)

print(out1.shape)
print(out2.shape)
print(out3.shape)
print(out4.shape)

torch.Size([8, 6, 60, 124])
torch.Size([8, 6, 30, 62])
torch.Size([8, 16, 26, 58])
torch.Size([8, 16, 13, 29])


In [ ]:
# 모델 만들기
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__() # 필수
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)

        out = self.layer3(out)
        return out

In [ ]:
model = CNN().to(device)

In [ ]:
# 직접 학습 전 오류 없는지 체크
test_input = torch.Tensor(8,3,64, 128).to(device)
test_prediction = model(test_input)
print(test_prediction)

tensor([[    nan,     nan],
        [    nan,     nan],
        [-0.0139, -0.0090],
        [    nan,     nan],
        [    nan,     nan],
        [-0.0139, -0.0090],
        [    nan,     nan],
        [    nan,     nan]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
criterion = nn.CrossEntropyLoss().to(device) # BinaryCrossEntropy 했을 때 오류남... why?
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
total_batch = len(data_loader)

for epoch in range(1, epochs+1):
    avg_cost = 0.0

    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)

        hypothesis = model(x)
        cost = criterion(hypothesis, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('Epoch: {:4d}, Cost: {:.6f}'.format(epoch, avg_cost))
print('Learning is finished')

Epoch:    1, Cost: 0.600702
Epoch:    2, Cost: 0.265038
Epoch:    3, Cost: 0.060216
Epoch:    4, Cost: 0.022007
Epoch:    5, Cost: 0.011409
Epoch:    6, Cost: 0.006790
Epoch:    7, Cost: 0.004557
Epoch:    8, Cost: 0.003332
Epoch:    9, Cost: 0.002450
Epoch:   10, Cost: 0.001895
Epoch:   11, Cost: 0.001521
Epoch:   12, Cost: 0.001254
Epoch:   13, Cost: 0.001049
Epoch:   14, Cost: 0.000857
Epoch:   15, Cost: 0.000746
Epoch:   16, Cost: 0.000635
Epoch:   17, Cost: 0.000546
Epoch:   18, Cost: 0.000479
Epoch:   19, Cost: 0.000426
Epoch:   20, Cost: 0.000380
Learning is finished


In [ ]:
# 성능 확인하기
trans2 = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])

test_data = dsets.ImageFolder(root='/content/drive/Othercomputers/내 노트북/인공지능/모두를 위한 딥러닝/custom_data/test_data', transform=trans2)

In [ ]:
test_sets = DataLoader(test_data, batch_size=len(test_data), shuffle=True)

In [ ]:
with torch.no_grad():
    for x,y in test_sets: # batch_size가 길이 전체이므로, x에 50개 다 들어있음. 즉 for문 한 번 수행
        x = x.to(device)
        y = y.to(device)

        prediction = model(x)
        correct_prediction = torch.argmax(prediction, 1) == y
        accuracy = correct_prediction.float().mean()
        print('Accuracy: {:.4f}'.format(accuracy.item()))

Accuracy: 1.0000


## 한 번 학습시킨 모델 저장하기

In [ ]:
# 한 번 학습시킨 모델을 저장하자
torch.save(model.state_dict(), '/model.pth')

In [ ]:
new_model = CNN().to(device)
new_model.load_state_dict(torch.load('/model.pth'))

<All keys matched successfully>

In [ ]:
# new model은 학습 안시킨 상태임
print((model.layer1[0].weight == new_model.layer1[0].weight).float().mean().item())

1.0
